In [1]:
#Genome Assemby- using synthetic reads from a 15894 long virus genome
#Based on Algorithms for DNA Sequencing course on Coursera
#Peter Sarvari

In [2]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [3]:
def make_dict(reads,k): 
    """ make a dictionary of all k-mers as keys and 
    reads that have those k-mers as values """
    suffixes = {}
    for read in reads:
        for i in range(0,len(read)-k+1):
            kmer = read[i:i+k]
            if kmer not in suffixes:
                suffixes[kmer] = [read]
            else:
                suffixes[kmer].append(read)
    return suffixes

In [4]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [5]:
def num_edges_with_length(a,suffixes,k):
    """ Given the suffix of 'a' what prefixes of other 
    reads have at least k overlap with it: in other words
    finding the out edges from a in the overlap graph """
    suffix = a[-k:]
    arr = suffixes[suffix] 
    pairs = []
    for element in arr:
        length = overlap(a,element,k)
        if length>=k: #if not zero is returned, then it should be at least k
            if not element == a:
                pairs.append((a,element,length))
    return pairs

In [6]:
def all_overlap_pairs_with_length(reads,suffixes,k):
    """ Finding the out edges for all reads in the overlap graph """
    all_pairs = []
    for read in reads:
        pairs = num_edges_with_length(read,suffixes,k)
        all_pairs = all_pairs + pairs
    return all_pairs

In [7]:
def pick_maximal_overlap_fast(all_pairs):
    """ Pick the edge that has the biggest value 
    (maximal overlap) in the overla graph """
    reada, readb = None, None
    best_olen = 0
    for pairs in all_pairs:
        if pairs[2] > best_olen:
            reada, readb = pairs[0], pairs[1]
            best_olen = pairs[2]
    return reada, readb, best_olen

In [17]:
def greedy_scs_fast(all_pairs,suffixes,reads,k):
    """ Greedy approach for assembly: merging the 
    nodes that have the biggest overlap between them """
    reada, readb, best_olen = pick_maximal_overlap_fast(all_pairs)
    #iter = 0
    while best_olen > 0:
        reads.remove(reada)
        reads.remove(readb)        
        reads.append(reada + readb[best_olen:])
        suffixes = make_dict(reads,k)
        all_pairs = all_overlap_pairs_with_length(reads,suffixes,k)
        reada, readb, best_olen = pick_maximal_overlap_fast(all_pairs) 
    return '.'.join(reads)

In [9]:
reads, _ = readFastq('ads1_week4_reads.fq')

In [18]:
suffixes = make_dict(reads,30)
all_pairs = all_overlap_pairs_with_length(reads, suffixes,30)
assembled_genome = greedy_scs_fast(all_pairs,suffixes,reads,30)

In [19]:
len(assembled_genome) #we got lucky and the greedy approach worked here! 

15894